In [67]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
market = pd.read_csv("market.csv")
market.index= market['date']
market = market.drop(columns=['Unnamed: 0'])
market= market.drop(columns=['SN'])
market = market.drop(columns=['date'])
market

,open,high,low,close,change,pct_change,turnover
date,,,,,,,
2022-07-07,2051.50,2079.30,2023.13,2027.32,-16.60,-0.81,1.806205e+09
2022-07-05,2078.63,2091.29,2026.37,2030.59,-46.08,-2.22,2.968485e+09
2022-07-04,2095.62,2148.16,2059.25,2076.67,-9.75,-0.47,4.349274e+09
2022-07-03,2028.79,2086.42,2021.00,2086.42,57.65,2.84,2.480474e+09
2022-06-30,1966.86,2037.64,1966.86,2037.64,74.93,3.82,2.458619e+09
...,...,...,...,...,...,...,...
2019-09-23,1139.92,1147.85,1137.87,1140.98,1.73,0.15,1.772790e+08
2019-08-26,1209.70,1210.24,1199.31,1203.13,-6.49,-0.54,3.805966e+08
2019-07-31,1259.53,1267.73,1257.91,1265.57,6.20,0.49,3.700020e+08


In [68]:
def standardize_data(df):
    scaler = StandardScaler()
    return pd.DataFrame(scaler.fit_transform(df), index=df.index, columns=df.columns)



def standardize_data(df):
    scaler = StandardScaler()
    return pd.DataFrame(scaler.fit_transform(df), index=df.index, columns=df.columns)


# Add technical indicators
def add_technical_indicators(df):
    # Moving Averages
    df['MA20'] = df['close'].rolling(window=20).mean()
    df['MA50'] = df['close'].rolling(window=50).mean()

    # Daily Returns
    df['Daily_Returns'] = df['pct_change']
    
    # Volatility (20-day rolling standard deviation of daily returns)
    df['Volatility'] = df['Daily_Returns'].rolling(window=20).std()



     # Exponential Moving Average (EMA)
    df['EMA20'] = df['close'].ewm(span=20, adjust=False).mean()
    df['EMA50'] = df['close'].ewm(span=50, adjust=False).mean()

    # Relative Strength Index (RSI)
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # Bollinger Bands (20-day SMA and 2 standard deviations)
    df['Bollinger_MA20'] = df['close'].rolling(window=20).mean()
    df['Bollinger_Upper'] = df['Bollinger_MA20'] + 2 * df['close'].rolling(window=20).std()
    df['Bollinger_Lower'] = df['Bollinger_MA20'] - 2 * df['close'].rolling(window=20).std()

    # MACD (Moving Average Convergence Divergence)
    df['EMA12'] = df['close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    

    return df



In [69]:
data_with_indicators = add_technical_indicators(market)

# Display the first few rows of the data with indicators
data_with_indicators

,open,high,low,close,change,pct_change,turnover,MA20,MA50,Daily_Returns,...,EMA20,EMA50,RSI,Bollinger_MA20,Bollinger_Upper,Bollinger_Lower,EMA12,EMA26,MACD,MACD_Signal
date,,,,,,,,,,,,,,,,,,,,,
2022-07-07,2051.50,2079.30,2023.13,2027.32,-16.60,-0.81,1.806205e+09,NaN,NaN,-0.81,...,2027.320000,2027.320000,NaN,NaN,NaN,NaN,2027.320000,2027.320000,0.000000,0.000000
2022-07-05,2078.63,2091.29,2026.37,2030.59,-46.08,-2.22,2.968485e+09,NaN,NaN,-2.22,...,2027.631429,2027.448235,NaN,NaN,NaN,NaN,2027.823077,2027.562222,0.260855,0.052171
2022-07-04,2095.62,2148.16,2059.25,2076.67,-9.75,-0.47,4.349274e+09,NaN,NaN,-0.47,...,2032.301769,2029.378501,NaN,NaN,NaN,NaN,2035.337988,2031.199835,4.138153,0.869367
2022-07-03,2028.79,2086.42,2021.00,2086.42,57.65,2.84,2.480474e+09,NaN,NaN,2.84,...,2037.455886,2031.615422,NaN,NaN,NaN,NaN,2043.196759,2035.290218,7.906541,2.276802
2022-06-30,1966.86,2037.64,1966.86,2037.64,74.93,3.82,2.458619e+09,NaN,NaN,3.82,...,2037.473421,2031.851680,NaN,NaN,NaN,NaN,2042.341873,2035.464276,6.877597,3.196961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-23,1139.92,1147.85,1137.87,1140.98,1.73,0.15,1.772790e+08,1206.0935,1327.9186,0.15,...,1205.878423,1312.246961,19.431369,1206.0935,1343.542832,1068.644168,1168.295769,1230.751842,-62.456073,-61.880442
2019-08-26,1209.70,1210.24,1199.31,1203.13,-6.49,-0.54,3.805966e+08,1199.9810,1323.9040,-0.54,...,1205.616669,1307.967865,39.865099,1199.9810,1325.445002,1074.516998,1173.654882,1228.705780,-55.050898,-60.514533
2019-07-31,1259.53,1267.73,1257.91,1265.57,6.20,0.49,3.700020e+08,1198.2285,1321.2898,0.49,...,1211.326510,1306.305203,58.592674,1198.2285,1318.651490,1077.805510,1187.795669,1231.436463,-43.640794,-57.139785


In [70]:
## remove and only making this  Date	Open	High	Low	Close	Adj Close	Volume	MA20	MA50	Daily Returns	Volatility
data_with_indicators= data_with_indicators.drop(columns=['change', 'pct_change', 'turnover'])

data_with_indicators.columns

Index(['open', 'high', 'low', 'close', 'MA20', 'MA50', 'Daily_Returns',
       'Volatility', 'EMA20', 'EMA50', 'RSI', 'Bollinger_MA20',
       'Bollinger_Upper', 'Bollinger_Lower', 'EMA12', 'EMA26', 'MACD',
       'MACD_Signal'],
      dtype='object')

In [71]:
data_with_indicators

,open,high,low,close,MA20,MA50,Daily_Returns,Volatility,EMA20,EMA50,RSI,Bollinger_MA20,Bollinger_Upper,Bollinger_Lower,EMA12,EMA26,MACD,MACD_Signal
date,,,,,,,,,,,,,,,,,,
2022-07-07,2051.50,2079.30,2023.13,2027.32,NaN,NaN,-0.81,NaN,2027.320000,2027.320000,NaN,NaN,NaN,NaN,2027.320000,2027.320000,0.000000,0.000000
2022-07-05,2078.63,2091.29,2026.37,2030.59,NaN,NaN,-2.22,NaN,2027.631429,2027.448235,NaN,NaN,NaN,NaN,2027.823077,2027.562222,0.260855,0.052171
2022-07-04,2095.62,2148.16,2059.25,2076.67,NaN,NaN,-0.47,NaN,2032.301769,2029.378501,NaN,NaN,NaN,NaN,2035.337988,2031.199835,4.138153,0.869367
2022-07-03,2028.79,2086.42,2021.00,2086.42,NaN,NaN,2.84,NaN,2037.455886,2031.615422,NaN,NaN,NaN,NaN,2043.196759,2035.290218,7.906541,2.276802
2022-06-30,1966.86,2037.64,1966.86,2037.64,NaN,NaN,3.82,NaN,2037.473421,2031.851680,NaN,NaN,NaN,NaN,2042.341873,2035.464276,6.877597,3.196961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-23,1139.92,1147.85,1137.87,1140.98,1206.0935,1327.9186,0.15,1.216437,1205.878423,1312.246961,19.431369,1206.0935,1343.542832,1068.644168,1168.295769,1230.751842,-62.456073,-61.880442
2019-08-26,1209.70,1210.24,1199.31,1203.13,1199.9810,1323.9040,-0.54,1.222417,1205.616669,1307.967865,39.865099,1199.9810,1325.445002,1074.516998,1173.654882,1228.705780,-55.050898,-60.514533
2019-07-31,1259.53,1267.73,1257.91,1265.57,1198.2285,1321.2898,0.49,1.178090,1211.326510,1306.305203,58.592674,1198.2285,1318.651490,1077.805510,1187.795669,1231.436463,-43.640794,-57.139785


In [72]:
features = data_with_indicators.drop(['close'], axis=1)
target = data_with_indicators['close']

## low datasets 

In [73]:
data_with_indicators_raw = data_with_indicators 

In [ ]:
data_with_indicators_cleaned= data_with_indicators.il
data_with_indicators_cleaned

In [75]:
data_with_indicators_raw[:]

,open,high,low,close,MA20,MA50,Daily_Returns,Volatility,EMA20,EMA50,RSI,Bollinger_MA20,Bollinger_Upper,Bollinger_Lower,EMA12,EMA26,MACD,MACD_Signal
date,,,,,,,,,,,,,,,,,,
2022-07-07,False,False,False,False,True,True,False,True,False,False,True,True,True,True,False,False,False,False
2022-07-05,False,False,False,False,True,True,False,True,False,False,True,True,True,True,False,False,False,False
2022-07-04,False,False,False,False,True,True,False,True,False,False,True,True,True,True,False,False,False,False
2022-07-03,False,False,False,False,True,True,False,True,False,False,True,True,True,True,False,False,False,False
2022-06-30,False,False,False,False,True,True,False,True,False,False,True,True,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-23,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2019-08-26,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2019-07-31,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [74]:
data_with_indicators

,open,high,low,close,MA20,MA50,Daily_Returns,Volatility,EMA20,EMA50,RSI,Bollinger_MA20,Bollinger_Upper,Bollinger_Lower,EMA12,EMA26,MACD,MACD_Signal
date,,,,,,,,,,,,,,,,,,
2022-07-07,2051.50,2079.30,2023.13,2027.32,NaN,NaN,-0.81,NaN,2027.320000,2027.320000,NaN,NaN,NaN,NaN,2027.320000,2027.320000,0.000000,0.000000
2022-07-05,2078.63,2091.29,2026.37,2030.59,NaN,NaN,-2.22,NaN,2027.631429,2027.448235,NaN,NaN,NaN,NaN,2027.823077,2027.562222,0.260855,0.052171
2022-07-04,2095.62,2148.16,2059.25,2076.67,NaN,NaN,-0.47,NaN,2032.301769,2029.378501,NaN,NaN,NaN,NaN,2035.337988,2031.199835,4.138153,0.869367
2022-07-03,2028.79,2086.42,2021.00,2086.42,NaN,NaN,2.84,NaN,2037.455886,2031.615422,NaN,NaN,NaN,NaN,2043.196759,2035.290218,7.906541,2.276802
2022-06-30,1966.86,2037.64,1966.86,2037.64,NaN,NaN,3.82,NaN,2037.473421,2031.851680,NaN,NaN,NaN,NaN,2042.341873,2035.464276,6.877597,3.196961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-23,1139.92,1147.85,1137.87,1140.98,1206.0935,1327.9186,0.15,1.216437,1205.878423,1312.246961,19.431369,1206.0935,1343.542832,1068.644168,1168.295769,1230.751842,-62.456073,-61.880442
2019-08-26,1209.70,1210.24,1199.31,1203.13,1199.9810,1323.9040,-0.54,1.222417,1205.616669,1307.967865,39.865099,1199.9810,1325.445002,1074.516998,1173.654882,1228.705780,-55.050898,-60.514533
2019-07-31,1259.53,1267.73,1257.91,1265.57,1198.2285,1321.2898,0.49,1.178090,1211.326510,1306.305203,58.592674,1198.2285,1318.651490,1077.805510,1187.795669,1231.436463,-43.640794,-57.139785


In [58]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader




# Split into features and target
features = market.drop(['close'], axis=1)
target = market['close']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# === Prepare LSTM sequences ===
def create_lstm_dataset(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10
X_train_lstm, y_train_lstm = create_lstm_dataset(X_train_scaled, y_train.values, time_steps)
X_test_lstm, y_test_lstm = create_lstm_dataset(X_test_scaled, y_test.values, time_steps)

# Convert to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train_lstm).float()
y_train_tensor = torch.from_numpy(y_train_lstm).float().unsqueeze(1)
X_test_tensor  = torch.from_numpy(X_test_lstm).float()
y_test_tensor  = torch.from_numpy(y_test_lstm).float().unsqueeze(1)

# DataLoader
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# === Define PyTorch LSTM Model ===
class LSTMModel(nn.Module):
    def __init__(self, num_features, hidden_size=100, num_layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=num_features, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True, 
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # last time step
        return self.fc(out)

num_features = X_train_lstm.shape[2]
model = LSTMModel(num_features)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# === Train the LSTM Model ===
epochs = 20
model.train()
for epoch in range(1, epochs + 1):
    epoch_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * xb.size(0)
    epoch_loss /= len(train_loader.dataset)
    print(f"Epoch {epoch}/{epochs}, Loss: {epoch_loss:.6f}")

# === LSTM Predictions ===
model.eval()
with torch.no_grad():
    lstm_preds = model(X_test_tensor).numpy().flatten()

# === Train Random Forest ===
rf_model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf_model.fit(X_train_scaled, y_train)
rf_preds_full = rf_model.predict(X_test_scaled)

# Align lengths for ensemble
min_len = min(len(lstm_preds), len(rf_preds_full))
ensemble_preds = (lstm_preds[:min_len] + rf_preds_full[:min_len]) / 2

# === Evaluate Ensemble ===
ensemble_mse = mean_squared_error(y_test_lstm[:min_len], ensemble_preds)
print("Ensemble Mean Squared Error:", ensemble_mse)


Epoch 1/20, Loss: nan


Epoch 2/20, Loss: nan
Epoch 3/20, Loss: nan
Epoch 4/20, Loss: nan
Epoch 5/20, Loss: nan
Epoch 6/20, Loss: nan
Epoch 7/20, Loss: nan
Epoch 8/20, Loss: nan
Epoch 9/20, Loss: nan
Epoch 10/20, Loss: nan
Epoch 11/20, Loss: nan
Epoch 12/20, Loss: nan
Epoch 13/20, Loss: nan
Epoch 14/20, Loss: nan
Epoch 15/20, Loss: nan
Epoch 16/20, Loss: nan
Epoch 17/20, Loss: nan
Epoch 18/20, Loss: nan
Epoch 19/20, Loss: nan
Epoch 20/20, Loss: nan


ValueError: Input contains NaN.

In [ ]:
1716806.4706014239


In [ ]:
data

,open,high,low,close,change,pct_change,turnover,date
date,,,,,,,,
2022-07-07,2051.50,2079.30,2023.13,2027.32,-16.60,-0.81,1.806205e+09,1.657152e+09
2022-07-05,2078.63,2091.29,2026.37,2030.59,-46.08,-2.22,2.968485e+09,1.656979e+09
2022-07-04,2095.62,2148.16,2059.25,2076.67,-9.75,-0.47,4.349274e+09,1.656893e+09
2022-07-03,2028.79,2086.42,2021.00,2086.42,57.65,2.84,2.480474e+09,1.656806e+09
2022-06-30,1966.86,2037.64,1966.86,2037.64,74.93,3.82,2.458619e+09,1.656547e+09
...,...,...,...,...,...,...,...,...
2019-09-23,1139.92,1147.85,1137.87,1140.98,1.73,0.15,1.772790e+08,1.569197e+09
2019-08-26,1209.70,1210.24,1199.31,1203.13,-6.49,-0.54,3.805966e+08,1.566778e+09
2019-07-31,1259.53,1267.73,1257.91,1265.57,6.20,0.49,3.700020e+08,1.564531e+09


In [ ]:
data['volume'] = data['turnover'] / data['close']
data

,open,high,low,close,change,pct_change,turnover,date,volume
date,,,,,,,,,
2022-07-07,2051.50,2079.30,2023.13,2027.32,-16.60,-0.81,1.806205e+09,1.657152e+09,8.909322e+05
2022-07-05,2078.63,2091.29,2026.37,2030.59,-46.08,-2.22,2.968485e+09,1.656979e+09,1.461883e+06
2022-07-04,2095.62,2148.16,2059.25,2076.67,-9.75,-0.47,4.349274e+09,1.656893e+09,2.094350e+06
2022-07-03,2028.79,2086.42,2021.00,2086.42,57.65,2.84,2.480474e+09,1.656806e+09,1.188866e+06
2022-06-30,1966.86,2037.64,1966.86,2037.64,74.93,3.82,2.458619e+09,1.656547e+09,1.206602e+06
...,...,...,...,...,...,...,...,...,...
2019-09-23,1139.92,1147.85,1137.87,1140.98,1.73,0.15,1.772790e+08,1.569197e+09,1.553744e+05
2019-08-26,1209.70,1210.24,1199.31,1203.13,-6.49,-0.54,3.805966e+08,1.566778e+09,3.163387e+05
2019-07-31,1259.53,1267.73,1257.91,1265.57,6.20,0.49,3.700020e+08,1.564531e+09,2.923600e+05


In [ ]:

# Define LSTM model using PyTorch
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=50, num_layers=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        output, _ = self.lstm(x)
        out = self.fc(output[:, -1, :])  # Get the last timestep's output
        return out

# Create sequences
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(n_steps, len(data)):
        X.append(data[i - n_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# Monte Carlo simulation
def monte_carlo_simulations(last_price, days, mu, sigma, simulations):
    results = np.zeros((days, simulations))
    for sim in range(simulations):
        prices = [last_price]
        for _ in range(days):
            prices.append(prices[-1] * np.exp((mu - 0.5 * sigma**2) + sigma * np.random.normal()))
        results[:, sim] = prices[1:]
    return results

# Calculate annualized return and std deviation
def calculate_annualized_params(df):
    log_returns = np.log(df['Close'] / df['Close'].shift(1))
    mu = np.mean(log_returns) * 252
    sigma = np.std(log_returns) * np.sqrt(252)
    return mu, sigma

# Training loop
def train_lstm_model(model, train_loader, val_loader, epochs=20, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs.squeeze(), y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch)
                loss = criterion(outputs.squeeze(), y_batch)
                val_loss += loss.item()
                
        print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")
    return model

# Main analysis function
def analyze_stocks(stock_list, n_steps, days_to_simulate, simulations):
    models_scalers = {}
    simulation_results = {}

    for stock in stock_list:
        df = pd.read_csv(f'{stock}.csv')
      
        features = ['open', 'high', 'low', 'close', 'volume']
        df['high'] = df['Max Price']
        df['low'] = df['Min Price']  
        df['close'] = df['Closing Price']
        df['volume'] = df['Traded Shares']
        df['open'] = df['Previous Closing'].fillna(df['Closing Price'])

        df['Daily Returns'] = df['close'].pct_change()







        scaler = StandardScaler()
        scaled_features = scaler.fit_transform(df[features])

        X, y = create_sequences(scaled_features, n_steps)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        # Convert to tensors
        X_train = torch.tensor(X_train, dtype=torch.float32)
        y_train = torch.tensor(y_train[:, 3], dtype=torch.float32)  # Predicting 'Close' price
        X_val = torch.tensor(X_val, dtype=torch.float32)
        y_val = torch.tensor(y_val[:, 3], dtype=torch.float32)

        train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
        val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32)

        model = LSTMModel(input_size=X_train.shape[2])
        model = train_lstm_model(model, train_loader, val_loader)

        models_scalers[stock] = (model, scaler)

        predicted_price = df['close'].iloc[-1]
        mu, sigma = calculate_annualized_params(df)

        simulated_prices = monte_carlo_simulations(predicted_price, days_to_simulate, mu, sigma, simulations)
        simulation_results[stock] = simulated_prices

    return simulation_results

# List of stock tickers
stock_list = ['HDL_filter', 'SHIVM_filter']
n_steps = 50
days_to_simulate = 252
simulations = 100

# Run analysis
simulation_results = analyze_stocks(stock_list, n_steps, days_to_simulate, simulations)

# Plot the results
for stock, sims in simulation_results.items():
    plt.figure(figsize=(10, 5))
    for i in range(min(sims.shape[1], 10)):  # Plot max 10 paths
        plt.plot(sims[:, i], label=f'Simulation {i}')
    plt.title(f'Monte Carlo Simulations for {stock}')
    plt.xlabel('Days')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.show()


Epoch 1, Train Loss: 0.6519, Val Loss: 0.5445
Epoch 2, Train Loss: 0.2841, Val Loss: 0.1290
Epoch 3, Train Loss: 0.1008, Val Loss: 0.0974
Epoch 4, Train Loss: 0.0831, Val Loss: 0.0575
Epoch 5, Train Loss: 0.0817, Val Loss: 0.0409
Epoch 6, Train Loss: 0.0578, Val Loss: 0.0379
Epoch 7, Train Loss: 0.0520, Val Loss: 0.0317
Epoch 8, Train Loss: 0.0498, Val Loss: 0.0341
Epoch 9, Train Loss: 0.0503, Val Loss: 0.0316
Epoch 10, Train Loss: 0.0521, Val Loss: 0.0297
Epoch 11, Train Loss: 0.0516, Val Loss: 0.0334
Epoch 12, Train Loss: 0.0472, Val Loss: 0.0274
Epoch 13, Train Loss: 0.0470, Val Loss: 0.0301
Epoch 14, Train Loss: 0.0451, Val Loss: 0.0275
Epoch 15, Train Loss: 0.0949, Val Loss: 0.0259
Epoch 16, Train Loss: 0.0446, Val Loss: 0.0274
Epoch 17, Train Loss: 0.0429, Val Loss: 0.0234
Epoch 18, Train Loss: 0.0409, Val Loss: 0.0262
Epoch 19, Train Loss: 0.0405, Val Loss: 0.0224
Epoch 20, Train Loss: 0.0395, Val Loss: 0.0233


KeyError: 'Close'

In [ ]:
data = pd.read_csv("HDL.csv")
data['open'] = data['Previous Closing'].fillna(data['Closing Price'])


In [ ]:
data

,Date,No. Of Transaction,Max Price,Min Price,Closing Price,Traded Shares,Amount,Previous Closing,Difference Rs.,open
0,2012-07-08,7.0,100.0,100.0,100.0,13810.0,1381000.0,100.0,0.0,100.0
1,2012-07-16,1.0,100.0,100.0,100.0,50.0,5000.0,100.0,0.0,100.0
2,2012-08-01,2.0,99.0,99.0,99.0,70.0,6930.0,100.0,-1.0,100.0
3,2012-09-18,1.0,100.0,100.0,100.0,20.0,2000.0,99.0,1.0,99.0
4,2013-02-05,2.0,98.0,98.0,98.0,330.0,32340.0,100.0,-2.0,100.0
...,...,...,...,...,...,...,...,...,...,...
1237,2022-07-04,1559.0,3800.0,3565.0,3600.0,45588.0,169051427.2,3625.0,-25.0,3625.0
1238,2022-07-05,1033.0,3648.0,3480.0,3512.0,28764.0,102165568.5,3600.0,-88.0,3600.0
1239,2022-07-06,797.0,3540.0,3390.8,3492.0,26686.0,92671753.8,3512.0,-20.0,3512.0
1240,2022-07-07,550.0,3650.0,3470.0,3490.0,16777.0,59008264.6,3492.0,-2.0,3492.0
